# Import Library and Check GPU Status

In [21]:
!python --version
import deeplabcut
import tensorflow as tf
import yaml
import os, glob
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

/bin/bash: /home/jialiulab/.conda/envs/DEEPLABCUT/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Python 3.8.13
Name: /physical_device:GPU:0   Type: GPU


# If this is a new project to be started, run these cells. Otherwise, skip these cells.

## *Project Definition*

In [3]:
# Specify
video_list = [r'/home/jialiulab/disk1/Hao_Shen_DLC/Videos/080522_1328.avi']

# Specify
config_path = deeplabcut.create_new_project('pupil', 'Hao Shen', video_list, \
                             working_directory = r'/home/jialiulab/disk1/Hao_Shen_DLC', copy_videos=False)

Project "/home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-17" already exists!


## *Other parameters to specify*

In [ ]:
videofile_path  = [config_path[:-12] + r'/videos']
VideoType = 'avi'
SHUF = 1

## *Modify the yaml file to configure project parameters*

In [19]:
with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    l = yaml.load(file, Loader=yaml.FullLoader)
    # print(l)
    l['bodyparts'] = ['left','right','top','bottom','lefttop', 'righttop', 'leftbottom','rightbottom', 
                      'left_pupil', 'right_pupil', 'top_pupil', 'bottom_pupil', 'lefttop_pupil',
                      'righttop_pupil', 'leftbottom_pupil', 'rightbottom_pupil']
    l['numframes2pick'] = 100
    l['TrainingFraction'] = [0.8]
    l['default_net_type'] = 'resnet_101'
    l['batch_size'] = 10
    print(l)
    
with open(config_path, 'w') as file:
    documents = yaml.dump(l, file)

{'Task': 'pupil', 'TrainingFraction': [0.8], 'alphavalue': 0.7, 'batch_size': 10, 'bodyparts': ['left', 'right', 'top', 'bottom'], 'colormap': 'rainbow', 'corner2move2': [50, 50], 'cropping': False, 'date': 'Sep17', 'default_augmenter': 'default', 'default_net_type': 'resnet_101', 'dotsize': 12, 'identity': None, 'iteration': 0, 'move2corner': True, 'multianimalproject': False, 'numframes2pick': 50, 'pcutoff': 0.6, 'project_path': '/home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-17', 'scorer': 'Hao Shen', 'skeleton': [['bodypart1', 'bodypart2'], ['objectA', 'bodypart3']], 'skeleton_color': 'black', 'snapshotindex': -1, 'start': 0, 'stop': 1, 'video_sets': {'/home/jialiulab/disk1/Hao_Shen_DLC/Videos/080522_1328.avi': {'crop': '0, 1440, 0, 1080'}}, 'x1': 0, 'x2': 640, 'y1': 277, 'y2': 624}


## *Extract Frame*

In [5]:
# deeplabcut.extract_frames(conﬁg_path,'automatic','kmeans',crop=False)
deeplabcut.extract_frames(conﬁg_path,'automatic','kmeans',crop=True)

Config file read successfully.
Do you want to extract (perhaps additional) frames for video: /home/jialiulab/disk1/Hao_Shen_DLC/Videos/080522_1328.avi ?


yes/no yes


Extracting frames based on uniform ...
Uniformly extracting of frames from 0.0  seconds to 1528.77  seconds.
Frames were successfully extracted, for the videos listed in the config.yaml file.

You can now label the frames using the function 'label_frames' (Note, you should label frames extracted from diverse videos (and many videos; we do not recommend training on single videos!)).


## *Label Frame*

In [6]:
deeplabcut.label_frames(conﬁg_path)

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


## *Check Labeled Frame (optional)*

In [10]:
# deeplabcut.check_labels(config_path)

## *Train test split*

In [20]:
deeplabcut.create_training_dataset(config_path, net_type='resnet_101', Shuffles=[SHUF], augmenter_type='imgaug')

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.8,
  1,
  (array([36, 39,  0, 47, 28, 15,  5, 11, 45, 44, 31, 34, 38, 30, 32, 49, 16,
          41,  2, 43, 14, 20, 13,  9, 25,  6, 24, 23, 17, 33,  1, 29, 18, 27,
          40, 46, 10,  7, 21,  3]),
   array([26, 48, 22,  4, 35,  8, 19, 37, 12, 42])))]

## *Modify the yaml file to configure model parameters*

In [50]:
l = [int(string[10:]) for string in os.listdir(config_path[:-12] + f'/dlc-models')]
l.sort()

model_cfg_path =config_path[:-12] + f'/dlc-models/iteration-{l[-1]}/' \
                + os.listdir(config_path[:-12] + f'/dlc-models/iteration-{l[-1]}')[0] +'/train/pose_cfg.yaml'
with open(model_cfg_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    l = yaml.load(file, Loader=yaml.FullLoader)
    print(l)
    l['batch_size'] = 2
    l['max_input_size'] = 1500
    l['net_type'] = 'resnet_101'
    l['multi_step'] = [[0.02, 10000], [0.01, 50000], [0.002, 200000]]
    
with open(model_cfg_path, 'w') as file:
    documents = yaml.dump(l, file)

{'all_joints': [[0], [1], [2], [3]], 'all_joints_names': ['left', 'right', 'top', 'bottom'], 'alpha_r': 0.02, 'apply_prob': 0.5, 'batch_size': 2, 'contrast': {'clahe': True, 'claheratio': 0.1, 'histeq': True, 'histeqratio': 0.1}, 'convolution': {'edge': False, 'emboss': {'alpha': [0.0, 1.0], 'strength': [0.5, 1.5]}, 'embossratio': 0.1, 'sharpen': False, 'sharpenratio': 0.3}, 'cropratio': 0.4, 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_pupilSep17/pupil_Hao Shen80shuffle1.mat', 'dataset_type': 'imgaug', 'decay_steps': 30000, 'display_iters': 1000, 'global_scale': 0.8, 'init_weights': '/home/jialiulab/.conda/envs/DEEPLABCUT/lib/python3.8/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_101.ckpt', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'location_refinement': True, 'locref_huber_loss': True, 'locref_loss_weight': 0.05, 'locref_stdev': 7.2801, 'lr_init': 0.0005, 'max_input_size': 1500, 'metadataset': 'training

## *Train*

In [5]:
deeplabcut.train_network(config_path, shuffle=SHUF, displayiters=100, saveiters=10000)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['left', 'right', 'top', 'bottom'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 2,
 'contrast': {'clahe': True,
              'claheratio': 0.1,
              'histeq': True,
              'histeqratio': 0.1},
 'convolution': {'edge': False,
                 'emboss': {'alpha': [0.0, 1.0], 'strength': [0.5, 1.5]},
                 'embossratio': 0.1,
                 'sharpen': False,
                 'sharpenratio': 0.3},
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_pupilSep17/pupil_Hao '
            'Shen80shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/jialiulab/.conda/envs/DEEPLABCUT/lib/python3.8/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_101.ckpt',
 'intermediate_supervision': False,
 'i

Selecting single-animal trainer
Batch Size is 2


/home/jialiulab/.conda/envs/DEEPLABCUT/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Loading ImageNet-pretrained resnet_101
Display_iters overwritten as 100
Save_iters overwritten as 10000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-17/dlc-models/iteration-0/pupilSep17-trainset80shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 2, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 'all_joints': [[0], [1], [2], [3]

iteration: 100 loss: 0.0365 lr: 0.002
iteration: 200 loss: 0.0176 lr: 0.002
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/jialiulab/.conda/envs/DEEPLABCUT/lib/python3.8/site-packages/tensorflow/python/client/session.py", line 1377, in _do_call
    return fn(*args)
  File "/home/jialiulab/.conda/envs/DEEPLABCUT/lib/python3.8/site-packages/tensorflow/python/client/session.py", line 1360, in _run_fn
    return self._call_tf_sessionrun(options, feed_dict, fetch_list,
  File "/home/jialiulab/.conda/envs/DEEPLABCUT/lib/python3.8/site-packages/tensorflow/python/client/session.py", line 1453, in _call_tf_sessionrun
    return tf_session.TF_SessionRun_wrapper(self._session, options, feed_dict,
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node fifo_queue_enqueue}}]]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jialiulab/.conda/envs/DEEPLABCUT

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


## *Analyze videos*

In [5]:
deeplabcut.analyze_videos(config_path, videofile_path, videotype=VideoType, shuffle=SHUF, batchsize=4)

Using snapshot-200000 for model /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/dlc-models/iteration-0/pupilSep15-trainset80shuffle1


/home/jialiulab/.conda/envs/DEEPLABCUT/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Analyzing all the videos in the directory...
Starting to analyze %  /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/videos/080522_1328.avi
Loading  /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/videos/080522_1328.avi
Duration of video [s]:  1528.68 , recorded with  34.72 fps!
Overall # of frames:  53079  found with (before cropping) frame dimensions:  1440 1080
Starting to extract posture


100%|█████████▉| 53000/53079 [41:32<00:03, 21.26it/s]

Saving results in /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet101_pupilSep15shuffle1_200000'

In [6]:
deeplabcut.filterpredictions(config_path, videofile_path, videotype=VideoType, shuffle=SHUF)

Analyzing all the videos in the directory...
Filtering with median model /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/videos/080522_1328.avi
Saving filtered csv poses!


In [10]:
deeplabcut.create_labeled_video(config_path,videofile_path, videotype=VideoType, shuffle=SHUF, filtered=True)

Analyzing all the videos in the directory...
Starting to process video: /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/videos/080522_1328.avi
Loading /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/videos/080522_1328.avi and data.
Duration of video [s]: 1528.68, recorded with 34.72 fps!
Overall # of frames: 53079 with cropped frame dimensions: 1440 1080
Generating frames and creating video.


100%|██████████| 53079/53079 [08:25<00:00, 104.91it/s]


# If continuously using an existing project (Transfer Learning to New Movies), run from here

## *Project Definition*

In [4]:
# Specify
project_name = 'pupil-Hao Shen-2022-09-17'
config_path = r"/home/jialiulab/disk1/Hao_Shen_DLC/" + project_name + "/config.yaml"

## *Other parameters to specify*

In [ ]:
videofile_path  = [config_path[:-12] + r'/videos']
VideoType = 'avi'
SHUF = 1

## *Add new videos*

In [6]:
deeplabcut.add_new_videos(config_path, ['/home/jialiulab/disk1/Hao_Shen_DLC/Videos/pupil_032722.avi'], copy_videos=False)

Attempting to create a symbolic link of the video ...
Created the symlink of /home/jialiulab/disk1/Hao_Shen_DLC/Videos/pupil_032722.avi to /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-17/videos/pupil_032722.avi
New videos were added to the project! Use the function 'extract_frames' to select frames for labeling.


## *Modify the yaml file to configure project parameters*
- only need to change the numframes2pick

In [8]:
with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    l = yaml.load(file, Loader=yaml.FullLoader)
    # print(l)
    l['numframes2pick'] = 50
    print(l)
    
with open(config_path, 'w') as file:
    documents = yaml.dump(l, file)

{'Task': 'pupil', 'scorer': 'Hao Shen', 'date': 'Sep17', 'multianimalproject': False, 'identity': None, 'project_path': '/home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-17', 'video_sets': {'/home/jialiulab/disk1/Hao_Shen_DLC/Videos/080522_1328.avi': {'crop': '0, 1440, 0, 1080'}, '/home/jialiulab/disk1/Hao_Shen_DLC/Videos/pupil_032722.avi': {'crop': '0, 1440, 0, 1080'}}, 'bodyparts': ['left', 'right', 'top', 'bottom'], 'start': 0, 'stop': 1, 'numframes2pick': 50, 'skeleton': [['bodypart1', 'bodypart2'], ['objectA', 'bodypart3']], 'skeleton_color': 'black', 'pcutoff': 0.6, 'dotsize': 12, 'alphavalue': 0.7, 'colormap': 'rainbow', 'TrainingFraction': [0.8], 'iteration': 0, 'default_net_type': 'resnet_101', 'default_augmenter': 'default', 'snapshotindex': -1, 'batch_size': 10, 'cropping': False, 'x1': 0, 'x2': 640, 'y1': 277, 'y2': 624, 'corner2move2': [50, 50], 'move2corner': True}
{'Task': 'pupil', 'scorer': 'Hao Shen', 'date': 'Sep17', 'multianimalproject': False, 'identity': N

## *Extract Frame*

In [9]:
deeplabcut.extract_frames(conﬁg_path,'automatic','kmeans',crop=True)

Config file read successfully.
Do you want to extract (perhaps additional) frames for video: /home/jialiulab/disk1/Hao_Shen_DLC/Videos/080522_1328.avi ?


yes/no no


Do you want to extract (perhaps additional) frames for video: /home/jialiulab/disk1/Hao_Shen_DLC/Videos/pupil_032722.avi ?


yes/no yes


Extracting frames based on uniform ...
Uniformly extracting of frames from 0.0  seconds to 570.54  seconds.
Frames were successfully extracted, for the videos listed in the config.yaml file.

You can now label the frames using the function 'label_frames' (Note, you should label frames extracted from diverse videos (and many videos; we do not recommend training on single videos!)).


## *Label Frame*

In [10]:
deeplabcut.label_frames(conﬁg_path)

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


## *Merge the newly labeled data with labeled data from previous rounds*

In [11]:
deeplabcut.merge_datasets(config_path)

Merged data sets and updated refinement iteration to 1.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## *Check Labeled Frame (optional)*

In [6]:
# deeplabcut.check_labels(config_path)

Creating images with labels by Hao Shen.


100%|██████████| 40/40 [00:08<00:00,  4.81it/s]

If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


## *Train test split*
- Should be performed on the newly merged dataset (i.e. with newly extracted and labeled frames)

In [12]:
deeplabcut.create_training_dataset(config_path, net_type='resnet_101', Shuffles=[SHUF], augmenter_type='imgaug')

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.8,
  1,
  (array([ 9, 40, 39, 19, 17, 16,  1, 31, 18, 10, 50, 54, 14,  2, 22, 24, 47,
           4, 36,  3, 11, 37, 21,  7, 33, 27, 43, 42, 25, 26, 12,  5, 23, 13,
          20, 35, 44, 34, 32,  8, 41,  6, 45, 51]),
   array([53, 15, 48, 52,  0, 38, 30, 49, 29, 46, 28])))]

## *Modify the yaml file to configure model parameters*
- Make sure to use the already trained model from the previous round (transfer learning)

In [ ]:
l_model = [int(string[10:]) for string in os.listdir(config_path[:-12] + f'/dlc-models')]
l_model.sort()
model_cfg_path =config_path[:-12] + f'/dlc-models/iteration-{l_model[-1]}/' \
                + os.listdir(config_path[:-12] + f'/dlc-models/iteration-{l_model[-1]}')[0] +'/train/pose_cfg.yaml'
folder_last_iter = config_path[:-12] + f'/dlc-models/iteration-{l_model[-2]}/' \
                + os.listdir(config_path[:-12] + f'/dlc-models/iteration-{l_model[-2]}')[0] +'/train'
os.chdir(folder_last_iter)
recent_model = -1
for file in glob.glob("*.meta"):
    if int(file[9:-5]) > recent_model:
        recent_model = int(file[9:-5])
        
with open(model_cfg_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    l = yaml.load(file, Loader=yaml.FullLoader)
    # print(l)
    l['batch_size'] = 2
    l['max_input_size'] = 1500
    l['net_type'] = 'resnet_101'
    l['multi_step'] = [[0.01, 10000], [0.002, 50000]]
    l['init_weights'] = folder_last_iter + '/' + f'snapshot-{recent_model}' 
    print(l)
with open(model_cfg_path, 'w') as file:
    documents = yaml.dump(l, file)

## *Train*

In [ ]:
deeplabcut.train_network(config_path, shuffle=SHUF, displayiters=100, saveiters=10000)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['left', 'right', 'top', 'bottom'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 2,
 'contrast': {'clahe': True,
              'claheratio': 0.1,
              'histeq': True,
              'histeqratio': 0.1},
 'convolution': {'edge': False,
                 'emboss': {'alpha': [0.0, 1.0], 'strength': [0.5, 1.5]},
                 'embossratio': 0.1,
                 'sharpen': False,
                 'sharpenratio': 0.3},
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-1/UnaugmentedDataSet_pupilSep17/pupil_Hao '
            'Shen80shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao '
                 'Shen-2022-09-17/dlc-models/iteration-0/pupilSep17-trainset80shuffle1/train/snapshot-200',
 'intermediate_supervision':

Selecting single-animal trainer
Batch Size is 2


/home/jialiulab/.conda/envs/DEEPLABCUT/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Loading already trained DLC with backbone: resnet_101
Display_iters overwritten as 100
Save_iters overwritten as 10000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-17/dlc-models/iteration-1/pupilSep17-trainset80shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 2, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 'all_joints': [[0]

iteration: 300 loss: 0.0167 lr: 0.01


## *Analyze videos*

In [ ]:
deeplabcut.analyze_videos(config_path, videofile_path,
                                        videotype=VideoType, shuffle=SHUF, batchsize=4)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15]],
 'all_joints_names': ['left',
                      'right',
                      'top',
                      'bottom',
                      'lefttop',
                      'righttop',
                      'leftbottom',
                      'rightbottom',
                      'left_pupil',
                      'right_pupil',
                      'top_pupil',
                      'bottom_pupil',
                      'lefttop_pupil',
                      'righttop_pupil',
                      'leftbottom_pupil',
                      'rightbottom_pupil'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-1

Using snapshot-250000 for model /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/dlc-models/iteration-1/pupilSep15-trainset80shuffle1
Analyzing all the videos in the directory...
Starting to analyze %  /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/videos/pupil_032722.avi
Loading  /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/videos/pupil_032722.avi
Duration of video [s]:  570.5 , recorded with  34.72 fps!
Overall # of frames:  19809  found with (before cropping) frame dimensions:  1440 1080
Starting to extract posture


100%|█████████▉| 19800/19809 [14:18<00:00, 23.05it/s]


Saving results in /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/videos...
Starting to analyze %  /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/videos/080522_1328.avi
Loading  /home/jialiulab/disk1/Hao_Shen_DLC/pupil-Hao Shen-2022-09-15/videos/080522_1328.avi
Duration of video [s]:  1528.68 , recorded with  34.72 fps!
Overall # of frames:  53079  found with (before cropping) frame dimensions:  1440 1080
Starting to extract posture


  1%|          | 530/53079 [00:33<37:50, 23.15it/s]

In [ ]:
deeplabcut.filterpredictions(config_path, [videofile_path+'080522_1328.avi', videofile_path+'pupil_032722.avi'], videotype=VideoType, shuffle=SHUF)

In [ ]:
deeplabcut.create_labeled_video(config_path,[videofile_path+'080522_1328.avi', videofile_path+'pupil_032722.avi'], videotype=VideoType, shuffle=SHUF, filtered=True)